In [10]:
from sys import path

import src
path.append('./src')
import pytest
from src.engine import Engine
from src.components.queues.base import *


In [7]:
a = Engine()

In [13]:
a.state.game_events.post(GAME_OVER)


In [14]:
a.update()

c:\Users\jason\workspaces\repos\jrl\.venv\Lib\site-packages\tcod\event.py:1210: RuntimeWarning: Events polled before SDL was initialized.
  warnings.warn(


In [16]:
a = set([1,2,3])

for i, x in enumerate(a):
    print(i, x)

0 1
1 2
2 3


In [ ]:
a.

TypeError: 'set' object is not subscriptable

In [3]:
from test.test_game_ai import *
test_game_over_event_handling()
test_game_ai_initialization()
test_game_ai_no_events()
test_game_ai_multiple_events()
test_game_ai_event_clearing()



In [4]:
from test.test_game_map import *
test_get_map_coords()
test_in_bounds()
test_is_traversable_transparent_visible_explored()
test_out_of_bounds_checks()
test_set_visible_and_reset()
test_update_explored()
test_width_height()
test_tiles_initialization()


In [5]:
from test.test_roster import *
test_roster_spawn()
test_roster_entity_blocked_locations()
test_roster_entity_collision()
test_roster_live_ai_actors()
test_roster_live_actors()
test_roster_get_entity_at_location()


Failed: Entity blocked locations did not function as expected